# A Fast Algorithm for Computing Distance Spectrum of Convolutional Codes
### by Mats Cedervall and Rolf Johannesson

In [1]:
%run state_transition_diagram.ipynb

def hamming_weight(matrix):
    weight = 0
    for i in range(matrix.nrows()):
        for j in range(matrix.ncols()):
            if matrix[i, j] != 0:
                weight += 1
    return weight

def degree(generator_matrix):
    # works only for row reduced matrices.
    row_degs = row_degrees(generator_matrix)
    degree = sum(row_degs)
    return degree
    
def get_coefficient_matrices(generator_matrix):
    # this function computes the matrices G_0,..., G_M from the generator matrix G(x)
    row_degs = row_degrees(generator_matrix)
    k = generator_matrix.nrows()
    n = generator_matrix.ncols()
    R = generator_matrix[0, 0].parent()
    x = R.0
    field = R.base()
    memory = max(row_degs)
    coeff_matrices = []
    for i in range(memory + 1):
        # for a polynomial f(x) = a_0 + a_1 x + ... + a_n x^n we get f(x) mod x^(i+1) = a_0 + ... + a_i x^i
        # therefore f(x) mod x^(i+1) - f(x) mod x^i = a_i x^i. Dividing by x^i we get a_i.
        # Similarly for the matrices.
        coeff_mat_poly = matrix(R, ((generator_matrix % x^(i+1)) - (generator_matrix % x^i)) / (x^i))
        coeff_mat = matrix(field, coeff_mat_poly)
        coeff_matrices.append(coeff_mat)
    return coeff_matrices

def make_sliding_matrix(generator_matrix, j):
    n = generator_matrix.ncols()
    coefficient_matrices = get_coefficient_matrices(generator_matrix)
    # make a long matrix (G_0,...,G_j)
    row_coefficient_matrices = block_matrix(1, j+1, coefficient_matrices[0:j+1])
    row_blocks = []
    for i in range(j+1):
        zero_mat = zero_matrix(generator_matrix.nrows(), n*i)
        # make the matrix (0, ..., 0, G_0,..., G_(j-i))
        row_block = block_matrix(1, 2, [[zero_mat, row_coefficient_matrices[:,:(j+1-i)*n]]], subdivide=False)
        row_blocks.append(row_block)
    # construct the sliding matrix from the matrices previously constructed.
    sliding_mat = block_matrix(j+1, 1, row_blocks, subdivide=False)
    return sliding_mat
    
def compute_distance_profile(generator_matrix):
    # this is a bruteforce method
    field = generator_matrix[0, 0].parent().base()
    q = field.cardinality()
    k = generator_matrix.nrows()
    n = generator_matrix.ncols()
    row_degs = row_degrees(generator_matrix)
    memory = max(row_degs)
    lowest_row_degree = min(row_degs)
    delta = degree(generator_matrix)
    distance_profile = []
    for j in range(memory + 1):
        # make the sliding matrix with first row block (G_0,..., G_j)
        sliding_matrix = make_sliding_matrix(generator_matrix, j)
        # use the generalized Singleton bound for rate 1/n codes as upper bound for the column distances.
        # we could also use another upper bound
        d_j = (n-k) * (delta//k + 1) + delta + 1
        # Consider all possible inputs. (This could be optimized since we also use inputs (u_0, ..., u_j)
        # with u_0 = 0 and we only later discard these inputs)
        # the input is a vector(u_0, ..., u_j) with u_i in F_q^k. Therefore the length is k*(j+1)
        len_input = k * (j + 1)
        # we make a list of lists of length len_input where each list contains all elements of the field.
        # This is used to generate all possible q^(k*(j+1)) inputs.
        inputs_to_be_combined = [[elt for elt in list(field)] for _ in range(len_input)]
        possible_inputs = list(itertools.product(*inputs_to_be_combined))
        possible_input_matrices = []
        for i in range(len(possible_inputs)):
            input_matrix = matrix(1, k*(j+1), possible_inputs[i])
            possible_input_matrices.append(input_matrix)
        for l in range(1, q^((j+1)*k)):
            # Calculate the output using the sliding matrix
            possible_inputs_l = possible_input_matrices[l]
            zero_col = vector([0 for _ in range(k)])
            output = possible_inputs_l * sliding_matrix
            # if the weight is smaller and u_0 is not the zero vector in F_q^k adjust d_j.
            if hamming_weight(output) < d_j and vector(possible_inputs_l[:,0]) != zero_col:
                d_j = hamming_weight(output)
        # append j-th column distance to distance profile
        distance_profile.append(d_j)
    return distance_profile

def adjust_stack(stack_list, W_0):
    adjusted_stack = []
    for S, W, m_vector in stack_list:
        if W - W_0 >= 0:
            # adjust the weight of a state by subtracting W_0.
            adjusted_stack.append((S, W - W_0, m_vector))
    return adjusted_stack

def determine_m_vector(state):
    m_vector = []
    for i in range(len(state)):
        m = 0
        if len(state[i]) == 0:
            m_vector.append(m)
            continue
        j = len(state[i]) - 1
        while state[i][j] == 0 and j >= 0:
            m += 1
            j -= 1
        m_vector.append(m)
    return m_vector

def new_m_vector(m_vector, inp):
    k = len(m_vector)
    m_vector_inp = [m_vector[i] + 1 if inp[i]== 0 else 0 for i in range(k)]
    return m_vector_inp

def max_row_degree_minus_m_vector(m_vector, row_degs):
    k = len(row_degs)
    return max([row_degs[i]- m_vector[i] for i in range(k)])
    
def fast_algorithm(generator_matrix, upper_bound="Singleton"):
    field = generator_matrix[0,0].parent().base()
    q = field.cardinality()
    k = generator_matrix.nrows()
    n = generator_matrix.ncols()
    polynomials = generator_matrix[0]
    try:
        row_degs, states = get_states(generator_matrix)
    except:
        raise
    state_transition_diagram, backward_diagram = make_diagram(generator_matrix, states, row_degs)
    memory = max(row_degs)
    lowest_row_degree = min(row_degs)
    delta = degree(generator_matrix)
    # set upper bound d on free distance using generalized Singleton bound.
    if upper_bound == "Singleton":
        d = (n - k) * (delta//k + 1) + delta + 1
    else:
        d = upper_bound
    zero_state = tuple([tuple([0 for _ in range(row_degs[i])]) for i in range(k)])
    next_states = backward_diagram[zero_state]
    stack_list = []
    # put all the possible next states except for the zero state on the stack.
    for new_state, multiedge in next_states.items():
        hamming_weight_output = hamming_weight(multiedge[0]["output"])
        if new_state != zero_state:
            if (d - hamming_weight_output >= 0):
                stack_list.append((new_state, d - hamming_weight_output, determine_m_vector(new_state)))
        else:
            # we don't consider the case of a completely zero row.
            if hamming_weight_output > 0:
                d = min(d, hamming_weight(multiedge[0]["output"]))
    state_count = 0
    distance_profile = compute_distance_profile(generator_matrix)
    found = False
    # make a loop that returns to F2 as long as the algorithm has not terminated.
    while not found:
        if len(stack_list) == 0:
            # found it
            return d, state_count
        S, W, m_vector = stack_list.pop()
        state_count += 1
        max_row_deg_minus_m = max_row_degree_minus_m_vector(m_vector, row_degs)
        
        possible_inputs = possibilities_inputs(generator_matrix, row_degs)
        nr_inputs = len(possible_inputs)
        stack_appendix = []
        # compute all the possible extensions. That is the state and its weight.
        for i in range(nr_inputs):
            inp = possible_inputs[i]
            S_inp = state_transition_diagram.nodes[S]["previous state"][inp]
            w_inp = hamming_weight(state_transition_diagram[S_inp][S][0]["output"])
            W_inp = W - w_inp
            m_vector_inp = new_m_vector(m_vector, inp)
            
            max_row_deg_minus_m_inp = max_row_degree_minus_m_vector(m_vector_inp, row_degs)
            # for case that a row degree is 0 we can get to the zero state without zero input.
            if S_inp == zero_state:
                if W_inp > 0:
                    # adjust the upper bound after the return to zero.
                    d = d - W_inp
                    W = W - W_inp
                    stack_list = adjust_stack(stack_list, W_inp)
                continue
            # the case that we don't skip
            if (W_inp >= distance_profile[max_row_deg_minus_m_inp-1] and W >= distance_profile[max_row_deg_minus_m-1]):
                # first put later ones that should be explored on the stack.
                stack_appendix.append((S_inp, W_inp, m_vector_inp))
        stack_list = stack_list + list(reversed(stack_appendix))
        
    return d, state_count